In [48]:
# Based on:
# https://www.youtube.com/watch?v=LOzcSuw3yOY
%load_ext Cython
import numpy as np
import functools
import matplotlib.pyplot as plt

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [49]:
nu = 0.25
a = 1
N = 10000
x = np.random.rand(N)
y = np.random.rand(N)

In [50]:
def original_f_vector(x, y, a, nu):
    """ Kernels with quadratic shape functions
        f has dimensions of (f=7, shapefunctions=3, n_obs)

        Classic form of: 
        arctan_x_minus_a = np.arctan((a - x) / y)
        arctan_x_plus_a = np.arctan((a + x) / y)

        but we have replaced these with the equaivalaent terms below to 
        avoid singularities at y = 0.  Singularities at x = +/- a still exist
    """

    arctan_x_minus_a = np.pi / 2 * np.sign(y / (a - x)) - np.arctan(y / (a - x))
    arctan_x_plus_a = np.pi / 2 * np.sign(y / (a + x)) - np.arctan(y / (a + x))

    f = np.zeros((7, 3, x.size))

    # f0
    f[0, 0, :] = (
        -1
        / 64
        * (
            6 * y ** 3 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a ** 3 * np.log(a ** 2 + 2 * a * x + x ** 2 + y ** 2)
            - 8 * a ** 3
            - 12 * a ** 2 * x
            + 12 * a * x ** 2
            - 12 * a * y ** 2
            + 6
            * (
                (2 * a * x - 3 * x ** 2) * arctan_x_plus_a
                + (2 * a * x - 3 * x ** 2) * arctan_x_minus_a
            )
            * y
            + 3
            * (a * x ** 2 - x ** 3 - (a - 3 * x) * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 3
            * (a * x ** 2 - x ** 3 - (a - 3 * x) * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[0, 1, :] = (
        1
        / 32
        * (
            6 * y ** 3 * (arctan_x_plus_a + arctan_x_minus_a)
            + a ** 3 * np.log(a ** 2 + 2 * a * x + x ** 2 + y ** 2)
            + a ** 3 * np.log(a ** 2 - 2 * a * x + x ** 2 + y ** 2)
            - 8 * a ** 3
            + 12 * a * x ** 2
            - 12 * a * y ** 2
            + 2
            * (
                (4 * a ** 2 - 9 * x ** 2) * arctan_x_plus_a
                + (4 * a ** 2 - 9 * x ** 2) * arctan_x_minus_a
            )
            * y
            + (4 * a ** 2 * x - 3 * x ** 3 + 9 * x * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - (4 * a ** 2 * x - 3 * x ** 3 + 9 * x * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[0, 2, :] = (
        -1
        / 64
        * (
            6 * y ** 3 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a ** 3 * np.log(a ** 2 - 2 * a * x + x ** 2 + y ** 2)
            - 8 * a ** 3
            + 12 * a ** 2 * x
            + 12 * a * x ** 2
            - 12 * a * y ** 2
            - 6
            * (
                (2 * a * x + 3 * x ** 2) * arctan_x_plus_a
                + (2 * a * x + 3 * x ** 2) * arctan_x_minus_a
            )
            * y
            - 3
            * (a * x ** 2 + x ** 3 - (a + 3 * x) * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + 3
            * (a * x ** 2 + x ** 3 - (a + 3 * x) * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    # f1
    f[1, 0, :] = (
        -3
        / 32
        * (
            3 * y ** 2 * (arctan_x_plus_a + arctan_x_minus_a)
            - (a - 3 * x) * y * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + (a - 3 * x) * y * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
            - 6 * a * y
            + (2 * a * x - 3 * x ** 2) * arctan_x_plus_a
            + (2 * a * x - 3 * x ** 2) * arctan_x_minus_a
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[1, 1, :] = (
        1
        / 16
        * (
            9 * y ** 2 * (arctan_x_plus_a + arctan_x_minus_a)
            + 9 * x * y * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 9 * x * y * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
            - 18 * a * y
            + (4 * a ** 2 - 9 * x ** 2) * arctan_x_plus_a
            + (4 * a ** 2 - 9 * x ** 2) * arctan_x_minus_a
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[1, 2, :] = (
        -3
        / 32
        * (
            3 * y ** 2 * (arctan_x_plus_a + arctan_x_minus_a)
            + (a + 3 * x) * y * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - (a + 3 * x) * y * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
            - 6 * a * y
            - (2 * a * x + 3 * x ** 2) * arctan_x_plus_a
            - (2 * a * x + 3 * x ** 2) * arctan_x_minus_a
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    # f2
    f[2, 0, :] = (
        3
        / 64
        * (
            8 * a ** 2
            - 12 * a * x
            - 4 * ((a - 3 * x) * arctan_x_plus_a + (a - 3 * x) * arctan_x_minus_a) * y
            - (2 * a * x - 3 * x ** 2 + 3 * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + (2 * a * x - 3 * x ** 2 + 3 * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[2, 1, :] = (
        1
        / 32
        * (
            36 * a * x
            - 36 * (x * arctan_x_plus_a + x * arctan_x_minus_a) * y
            + (4 * a ** 2 - 9 * x ** 2 + 9 * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - (4 * a ** 2 - 9 * x ** 2 + 9 * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    f[2, 2, :] = (
        -3
        / 64
        * (
            8 * a ** 2
            + 12 * a * x
            - 4 * ((a + 3 * x) * arctan_x_plus_a + (a + 3 * x) * arctan_x_minus_a) * y
            - (2 * a * x + 3 * x ** 2 - 3 * y ** 2)
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + (2 * a * x + 3 * x ** 2 - 3 * y ** 2)
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (np.pi * a ** 2 * nu - np.pi * a ** 2)
    )

    # f3
    f[3, 0, :] = (
        3
        / 32
        * (
            4 * a ** 2 * y ** 3
            - 2
            * ((a - 3 * x) * arctan_x_plus_a + (a - 3 * x) * arctan_x_minus_a)
            * y ** 4
            - 4
            * (
                (a ** 3 - 3 * a ** 2 * x + a * x ** 2 - 3 * x ** 3) * arctan_x_plus_a
                + (a ** 3 - 3 * a ** 2 * x + a * x ** 2 - 3 * x ** 3) * arctan_x_minus_a
            )
            * y ** 2
            + 4 * (a ** 4 - 3 * a ** 3 * x + a ** 2 * x ** 2) * y
            - 2
            * (
                a ** 5
                - 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                + 6 * a ** 2 * x ** 3
                + a * x ** 4
                - 3 * x ** 5
            )
            * arctan_x_plus_a
            - 2
            * (
                a ** 5
                - 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                + 6 * a ** 2 * x ** 3
                + a * x ** 4
                - 3 * x ** 5
            )
            * arctan_x_minus_a
            - 3
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + 3
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    f[3, 1, :] = (
        1
        / 16
        * (
            20 * a ** 3 * x * y
            - 18 * (x * arctan_x_plus_a + x * arctan_x_minus_a) * y ** 4
            - 36
            * (
                (a ** 2 * x + x ** 3) * arctan_x_plus_a
                + (a ** 2 * x + x ** 3) * arctan_x_minus_a
            )
            * y ** 2
            - 18 * (a ** 4 * x - 2 * a ** 2 * x ** 3 + x ** 5) * arctan_x_plus_a
            - 18 * (a ** 4 * x - 2 * a ** 2 * x ** 3 + x ** 5) * arctan_x_minus_a
            + 9
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 9
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    f[3, 2, :] = (
        -3
        / 32
        * (
            4 * a ** 2 * y ** 3
            - 2
            * ((a + 3 * x) * arctan_x_plus_a + (a + 3 * x) * arctan_x_minus_a)
            * y ** 4
            - 4
            * (
                (a ** 3 + 3 * a ** 2 * x + a * x ** 2 + 3 * x ** 3) * arctan_x_plus_a
                + (a ** 3 + 3 * a ** 2 * x + a * x ** 2 + 3 * x ** 3) * arctan_x_minus_a
            )
            * y ** 2
            + 4 * (a ** 4 + 3 * a ** 3 * x + a ** 2 * x ** 2) * y
            - 2
            * (
                a ** 5
                + 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                - 6 * a ** 2 * x ** 3
                + a * x ** 4
                + 3 * x ** 5
            )
            * arctan_x_plus_a
            - 2
            * (
                a ** 5
                + 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                - 6 * a ** 2 * x ** 3
                + a * x ** 4
                + 3 * x ** 5
            )
            * arctan_x_minus_a
            + 3
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 3
            * (
                y ** 5
                + 2 * (a ** 2 + x ** 2) * y ** 3
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * y
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    # f4
    f[4, 0, :] = (
        3
        / 32
        * (
            6 * y ** 5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a ** 5
            - 4 * a ** 4 * x
            + 18 * a ** 3 * x ** 2
            + 4 * a ** 2 * x ** 3
            - 12 * a * x ** 4
            - 12 * a * y ** 4
            + 12
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 3
            - 2 * (9 * a ** 3 - 2 * a ** 2 * x + 12 * a * x ** 2) * y ** 2
            + 6
            * (
                (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_plus_a
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_minus_a
            )
            * y
            - (
                a ** 5
                - 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                + 6 * a ** 2 * x ** 3
                + a * x ** 4
                - 3 * x ** 5
                + (a - 3 * x) * y ** 4
                + 2 * (a ** 3 - 3 * a ** 2 * x + a * x ** 2 - 3 * x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + (
                a ** 5
                - 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                + 6 * a ** 2 * x ** 3
                + a * x ** 4
                - 3 * x ** 5
                + (a - 3 * x) * y ** 4
                + 2 * (a ** 3 - 3 * a ** 2 * x + a * x ** 2 - 3 * x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    f[4, 1, :] = (
        -1
        / 16
        * (
            18 * y ** 5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 26 * a ** 5
            + 62 * a ** 3 * x ** 2
            - 36 * a * x ** 4
            - 36 * a * y ** 4
            + 36
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 3
            - 2 * (31 * a ** 3 + 36 * a * x ** 2) * y ** 2
            + 18
            * (
                (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_plus_a
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_minus_a
            )
            * y
            + 9
            * (
                a ** 4 * x
                - 2 * a ** 2 * x ** 3
                + x ** 5
                + x * y ** 4
                + 2 * (a ** 2 * x + x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 9
            * (
                a ** 4 * x
                - 2 * a ** 2 * x ** 3
                + x ** 5
                + x * y ** 4
                + 2 * (a ** 2 * x + x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    f[4, 2, :] = (
        3
        / 32
        * (
            6 * y ** 5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a ** 5
            + 4 * a ** 4 * x
            + 18 * a ** 3 * x ** 2
            - 4 * a ** 2 * x ** 3
            - 12 * a * x ** 4
            - 12 * a * y ** 4
            + 12
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 3
            - 2 * (9 * a ** 3 + 2 * a ** 2 * x + 12 * a * x ** 2) * y ** 2
            + 6
            * (
                (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_plus_a
                + (a ** 4 - 2 * a ** 2 * x ** 2 + x ** 4) * arctan_x_minus_a
            )
            * y
            + (
                a ** 5
                + 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                - 6 * a ** 2 * x ** 3
                + a * x ** 4
                + 3 * x ** 5
                + (a + 3 * x) * y ** 4
                + 2 * (a ** 3 + 3 * a ** 2 * x + a * x ** 2 + 3 * x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - (
                a ** 5
                + 3 * a ** 4 * x
                - 2 * a ** 3 * x ** 2
                - 6 * a ** 2 * x ** 3
                + a * x ** 4
                + 3 * x ** 5
                + (a + 3 * x) * y ** 4
                + 2 * (a ** 3 + 3 * a ** 2 * x + a * x ** 2 + 3 * x ** 3) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 6 * nu
            - np.pi * a ** 6
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 4
            - 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            + 2
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 2
        )
    )

    # f5
    f[5, 0, :] = (
        3
        / 32
        * (
            8 * a ** 8
            - 24 * a ** 7 * x
            - 16 * a ** 6 * x ** 2
            + 60 * a ** 5 * x ** 3
            + 8 * a ** 4 * x ** 4
            - 48 * a ** 3 * x ** 5
            + 12 * a * x ** 7
            + 12 * a * x * y ** 6
            + 4 * (2 * a ** 4 + 12 * a ** 3 * x + 9 * a * x ** 3) * y ** 4
            + 4
            * (4 * a ** 6 + 3 * a ** 5 * x - 12 * a ** 4 * x ** 2 + 9 * a * x ** 5)
            * y ** 2
            - 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )

    f[5, 1, :] = (
        1
        / 16
        * (
            56 * a ** 7 * x
            - 148 * a ** 5 * x ** 3
            + 128 * a ** 3 * x ** 5
            - 36 * a * x ** 7
            - 36 * a * x * y ** 6
            - 12 * (8 * a ** 3 * x + 9 * a * x ** 3) * y ** 4
            - 4 * (a ** 5 * x - 8 * a ** 3 * x ** 3 + 27 * a * x ** 5) * y ** 2
            + 9
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 9
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )

    f[5, 2, :] = (
        -3
        / 32
        * (
            8 * a ** 8
            + 24 * a ** 7 * x
            - 16 * a ** 6 * x ** 2
            - 60 * a ** 5 * x ** 3
            + 8 * a ** 4 * x ** 4
            + 48 * a ** 3 * x ** 5
            - 12 * a * x ** 7
            - 12 * a * x * y ** 6
            + 4 * (2 * a ** 4 - 12 * a ** 3 * x - 9 * a * x ** 3) * y ** 4
            + 4
            * (4 * a ** 6 - 3 * a ** 5 * x - 12 * a ** 4 * x ** 2 - 9 * a * x ** 5)
            * y ** 2
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 + 2 * a * x + x ** 2 + y ** 2))
            - 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
                + y ** 8
                + 4 * (a ** 2 + x ** 2) * y ** 6
                + 2 * (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * y ** 4
                + 4 * (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * y ** 2
            )
            * np.log(abs(a ** 2 - 2 * a * x + x ** 2 + y ** 2))
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )

    # f6
    f[6, 0, :] = (
        -3
        / 16
        * (
            3 * y ** 8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a * y ** 7
            + 12
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 6
            - 6 * (4 * a ** 3 + 3 * a * x ** 2) * y ** 5
            + 6
            * (
                (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_plus_a
                + (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_minus_a
            )
            * y ** 4
            - 2 * (15 * a ** 5 - 8 * a ** 4 * x + 9 * a * x ** 4) * y ** 3
            + 12
            * (
                (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * arctan_x_plus_a
                + (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6)
                * arctan_x_minus_a
            )
            * y ** 2
            - 2
            * (
                6 * a ** 7
                - 8 * a ** 6 * x
                + 3 * a ** 5 * x ** 2
                + 8 * a ** 4 * x ** 3
                - 12 * a ** 3 * x ** 4
                + 3 * a * x ** 6
            )
            * y
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_plus_a
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_minus_a
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )

    f[6, 1, :] = (
        1
        / 8
        * (
            9 * y ** 8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 18 * a * y ** 7
            + 36
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 6
            - 2 * (32 * a ** 3 + 27 * a * x ** 2) * y ** 5
            + 18
            * (
                (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_plus_a
                + (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_minus_a
            )
            * y ** 4
            - 2 * (37 * a ** 5 + 8 * a ** 3 * x ** 2 + 27 * a * x ** 4) * y ** 3
            + 36
            * (
                (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * arctan_x_plus_a
                + (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6)
                * arctan_x_minus_a
            )
            * y ** 2
            - 2
            * (14 * a ** 7 + a ** 5 * x ** 2 - 24 * a ** 3 * x ** 4 + 9 * a * x ** 6)
            * y
            + 9
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_plus_a
            + 9
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_minus_a
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )

    f[6, 2, :] = (
        -3
        / 16
        * (
            3 * y ** 8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a * y ** 7
            + 12
            * (
                (a ** 2 + x ** 2) * arctan_x_plus_a
                + (a ** 2 + x ** 2) * arctan_x_minus_a
            )
            * y ** 6
            - 6 * (4 * a ** 3 + 3 * a * x ** 2) * y ** 5
            + 6
            * (
                (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_plus_a
                + (3 * a ** 4 + 2 * a ** 2 * x ** 2 + 3 * x ** 4) * arctan_x_minus_a
            )
            * y ** 4
            - 2 * (15 * a ** 5 + 8 * a ** 4 * x + 9 * a * x ** 4) * y ** 3
            + 12
            * (
                (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6) * arctan_x_plus_a
                + (a ** 6 - a ** 4 * x ** 2 - a ** 2 * x ** 4 + x ** 6)
                * arctan_x_minus_a
            )
            * y ** 2
            - 2
            * (
                6 * a ** 7
                + 8 * a ** 6 * x
                + 3 * a ** 5 * x ** 2
                - 8 * a ** 4 * x ** 3
                - 12 * a ** 3 * x ** 4
                + 3 * a * x ** 6
            )
            * y
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_plus_a
            + 3
            * (
                a ** 8
                - 4 * a ** 6 * x ** 2
                + 6 * a ** 4 * x ** 4
                - 4 * a ** 2 * x ** 6
                + x ** 8
            )
            * arctan_x_minus_a
        )
        / (
            np.pi * a ** 10 * nu
            - np.pi * a ** 10
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 8
            + (np.pi * a ** 2 * nu - np.pi * a ** 2) * y ** 8
            - 4 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 6
            + 4
            * (
                np.pi * a ** 4 * nu
                - np.pi * a ** 4
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 2
            )
            * y ** 6
            + 6 * (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 4
            + 2
            * (
                3 * np.pi * a ** 6 * nu
                - 3 * np.pi * a ** 6
                + 3 * (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 4
                + 2 * (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 2
            )
            * y ** 4
            - 4 * (np.pi * a ** 8 * nu - np.pi * a ** 8) * x ** 2
            + 4
            * (
                np.pi * a ** 8 * nu
                - np.pi * a ** 8
                + (np.pi * a ** 2 * nu - np.pi * a ** 2) * x ** 6
                - (np.pi * a ** 4 * nu - np.pi * a ** 4) * x ** 4
                - (np.pi * a ** 6 * nu - np.pi * a ** 6) * x ** 2
            )
            * y ** 2
        )
    )
    return f

In [53]:
def original_f_vector_sub(x, y, a, nu):
    """ Kernels with quadratic shape functions
        f has dimensions of (f=7, shapefunctions=3, n_obs)

        Classic form of: 
        arctan_x_minus_a = np.arctan((a - x) / y)
        arctan_x_plus_a = np.arctan((a + x) / y)

        but we have replaced these with the equivalent terms below to avoid
      singularities at y = 0.  Singularities at x = +/- a still exist
    """

    pi = np.pi
    
    arctan_x_minus_a = pi / 2 * np.sign(y / (a - x)) - np.arctan(y / (a - x))
    arctan_x_plus_a = pi / 2 * np.sign(y / (a + x)) - np.arctan(y / (a + x))

    # Substitutions
    x2 = x ** 2
    x3 = x ** 3
    x4 = x ** 4
    x5 = x ** 5
    x6 = x ** 6
    x7 = x ** 7
    x8 = x ** 8

    y2 = y ** 2
    y3 = y ** 3
    y4 = y ** 4
    y5 = y ** 5
    y6 = y ** 6
    y7 = y ** 7
    y8 = y ** 8
    y9 = y ** 9
    y10 = y ** 10

    a2 = a ** 2
    a3 = a ** 3
    a4 = a ** 4
    a5 = a ** 5
    a6 = a ** 6
    a7 = a ** 7
    a8 = a ** 8
    a10 = a ** 10


    f = np.zeros((7, 3, x.size))

    # f0
    f[0, 0, :] = (
        -1
        / 64
        * (
            6 * y3 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a3 * np.log(a2 + 2 * a * x + x2+ y2)
            - 8 * a3
            - 12 * a2 * x
            + 12 * a * x2
            - 12 * a * y2
            + 6
            * (
                (2 * a * x - 3 * x2) * arctan_x_plus_a
                + (2 * a * x - 3 * x2) * arctan_x_minus_a
            )
            * y
            + 3
            * (a * x2- x3- (a - 3 * x) * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 3
            * (a * x2- x3- (a - 3 * x) * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[0, 1, :] = (
        1
        / 32
        * (
            6 * y3 * (arctan_x_plus_a + arctan_x_minus_a)
            + a3 * np.log(a2 + 2 * a * x + x2+ y2)
            + a3 * np.log(a2 - 2 * a * x + x2+ y2)
            - 8 * a3
            + 12 * a * x2
            - 12 * a * y2
            + 2
            * (
                (4 * a2 - 9 * x2) * arctan_x_plus_a
                + (4 * a2 - 9 * x2) * arctan_x_minus_a
            )
            * y
            + (4 * a2 * x - 3 * x3+ 9 * x * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - (4 * a2 * x - 3 * x3+ 9 * x * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[0, 2, :] = (
        -1
        / 64
        * (
            6 * y3 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a3 * np.log(a2 - 2 * a * x + x2+ y2)
            - 8 * a3
            + 12 * a2 * x
            + 12 * a * x2
            - 12 * a * y2
            - 6
            * (
                (2 * a * x + 3 * x2) * arctan_x_plus_a
                + (2 * a * x + 3 * x2) * arctan_x_minus_a
            )
            * y
            - 3
            * (a * x2+ x3- (a + 3 * x) * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + 3
            * (a * x2+ x3- (a + 3 * x) * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    # f1
    f[1, 0, :] = (
        -3
        / 32
        * (
            3 * y2 * (arctan_x_plus_a + arctan_x_minus_a)
            - (a - 3 * x) * y * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + (a - 3 * x) * y * np.log(abs(a2 - 2 * a * x + x2+ y2))
            - 6 * a * y
            + (2 * a * x - 3 * x2) * arctan_x_plus_a
            + (2 * a * x - 3 * x2) * arctan_x_minus_a
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[1, 1, :] = (
        1
        / 16
        * (
            9 * y2 * (arctan_x_plus_a + arctan_x_minus_a)
            + 9 * x * y * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 9 * x * y * np.log(abs(a2 - 2 * a * x + x2+ y2))
            - 18 * a * y
            + (4 * a2 - 9 * x2) * arctan_x_plus_a
            + (4 * a2 - 9 * x2) * arctan_x_minus_a
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[1, 2, :] = (
        -3
        / 32
        * (
            3 * y2 * (arctan_x_plus_a + arctan_x_minus_a)
            + (a + 3 * x) * y * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - (a + 3 * x) * y * np.log(abs(a2 - 2 * a * x + x2+ y2))
            - 6 * a * y
            - (2 * a * x + 3 * x2) * arctan_x_plus_a
            - (2 * a * x + 3 * x2) * arctan_x_minus_a
        )
        / (pi * a2 * nu - pi * a2)
    )

    # f2
    f[2, 0, :] = (
        3
        / 64
        * (
            8 * a2
            - 12 * a * x
            - 4 * ((a - 3 * x) * arctan_x_plus_a + (a - 3 * x) * arctan_x_minus_a) * y
            - (2 * a * x - 3 * x2+ 3 * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + (2 * a * x - 3 * x2+ 3 * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[2, 1, :] = (
        1
        / 32
        * (
            36 * a * x
            - 36 * (x * arctan_x_plus_a + x * arctan_x_minus_a) * y
            + (4 * a2 - 9 * x2+ 9 * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - (4 * a2 - 9 * x2+ 9 * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    f[2, 2, :] = (
        -3
        / 64
        * (
            8 * a2
            + 12 * a * x
            - 4 * ((a + 3 * x) * arctan_x_plus_a + (a + 3 * x) * arctan_x_minus_a) * y
            - (2 * a * x + 3 * x2- 3 * y2)
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + (2 * a * x + 3 * x2- 3 * y2)
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (pi * a2 * nu - pi * a2)
    )

    # f3
    f[3, 0, :] = (
        3
        / 32
        * (
            4 * a2 * y3
            - 2
            * ((a - 3 * x) * arctan_x_plus_a + (a - 3 * x) * arctan_x_minus_a)
            * y4
            - 4
            * (
                (a3 - 3 * a2 * x + a * x2- 3 * x3) * arctan_x_plus_a
                + (a3 - 3 * a2 * x + a * x2- 3 * x3) * arctan_x_minus_a
            )
            * y2
            + 4 * (a4 - 3 * a3 * x + a2 * x2) * y
            - 2
            * (
                a5
                - 3 * a4 * x
                - 2 * a3 * x2
                + 6 * a2 * x3
                + a * x4
                - 3 * x5
            )
            * arctan_x_plus_a
            - 2
            * (
                a5
                - 3 * a4 * x
                - 2 * a3 * x2
                + 6 * a2 * x3
                + a * x4
                - 3 * x5
            )
            * arctan_x_minus_a
            - 3
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + 3
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    f[3, 1, :] = (
        1
        / 16
        * (
            20 * a3 * x * y
            - 18 * (x * arctan_x_plus_a + x * arctan_x_minus_a) * y4
            - 36
            * (
                (a2 * x + x3) * arctan_x_plus_a
                + (a2 * x + x3) * arctan_x_minus_a
            )
            * y2
            - 18 * (a4 * x - 2 * a2 * x3+ x5) * arctan_x_plus_a
            - 18 * (a4 * x - 2 * a2 * x3+ x5) * arctan_x_minus_a
            + 9
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 9
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    f[3, 2, :] = (
        -3
        / 32
        * (
            4 * a2 * y3
            - 2
            * ((a + 3 * x) * arctan_x_plus_a + (a + 3 * x) * arctan_x_minus_a)
            * y4
            - 4
            * (
                (a3 + 3 * a2 * x + a * x2+ 3 * x3) * arctan_x_plus_a
                + (a3 + 3 * a2 * x + a * x2+ 3 * x3) * arctan_x_minus_a
            )
            * y2
            + 4 * (a4 + 3 * a3 * x + a2 * x2) * y
            - 2
            * (
                a5
                + 3 * a4 * x
                - 2 * a3 * x2
                - 6 * a2 * x3
                + a * x4
                + 3 * x5
            )
            * arctan_x_plus_a
            - 2
            * (
                a5
                + 3 * a4 * x
                - 2 * a3 * x2
                - 6 * a2 * x3
                + a * x4
                + 3 * x5
            )
            * arctan_x_minus_a
            + 3
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 3
            * (
                y5
                + 2 * (a2 + x2) * y3
                + (a4 - 2 * a2 * x2+ x4) * y
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    # f4
    f[4, 0, :] = (
        3
        / 32
        * (
            6 * y5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a5
            - 4 * a4 * x
            + 18 * a3 * x2
            + 4 * a2 * x3
            - 12 * a * x4
            - 12 * a * y4
            + 12
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y3
            - 2 * (9 * a3 - 2 * a2 * x + 12 * a * x2) * y2
            + 6
            * (
                (a4 - 2 * a2 * x2+ x4) * arctan_x_plus_a
                + (a4 - 2 * a2 * x2+ x4) * arctan_x_minus_a
            )
            * y
            - (
                a5
                - 3 * a4 * x
                - 2 * a3 * x2
                + 6 * a2 * x3
                + a * x4
                - 3 * x5
                + (a - 3 * x) * y4
                + 2 * (a3 - 3 * a2 * x + a * x2- 3 * x3) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + (
                a5
                - 3 * a4 * x
                - 2 * a3 * x2
                + 6 * a2 * x3
                + a * x4
                - 3 * x5
                + (a - 3 * x) * y4
                + 2 * (a3 - 3 * a2 * x + a * x2- 3 * x3) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    f[4, 1, :] = (
        -1
        / 16
        * (
            18 * y5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 26 * a5
            + 62 * a3 * x2
            - 36 * a * x4
            - 36 * a * y4
            + 36
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y3
            - 2 * (31 * a3 + 36 * a * x2) * y2
            + 18
            * (
                (a4 - 2 * a2 * x2+ x4) * arctan_x_plus_a
                + (a4 - 2 * a2 * x2+ x4) * arctan_x_minus_a
            )
            * y
            + 9
            * (
                a4 * x
                - 2 * a2 * x3
                + x5
                + x * y4
                + 2 * (a2 * x + x3) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 9
            * (
                a4 * x
                - 2 * a2 * x3
                + x5
                + x * y4
                + 2 * (a2 * x + x3) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    f[4, 2, :] = (
        3
        / 32
        * (
            6 * y5 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a5
            + 4 * a4 * x
            + 18 * a3 * x2
            - 4 * a2 * x3
            - 12 * a * x4
            - 12 * a * y4
            + 12
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y3
            - 2 * (9 * a3 + 2 * a2 * x + 12 * a * x2) * y2
            + 6
            * (
                (a4 - 2 * a2 * x2+ x4) * arctan_x_plus_a
                + (a4 - 2 * a2 * x2+ x4) * arctan_x_minus_a
            )
            * y
            + (
                a5
                + 3 * a4 * x
                - 2 * a3 * x2
                - 6 * a2 * x3
                + a * x4
                + 3 * x5
                + (a + 3 * x) * y4
                + 2 * (a3 + 3 * a2 * x + a * x2+ 3 * x3) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - (
                a5
                + 3 * a4 * x
                - 2 * a3 * x2
                - 6 * a2 * x3
                + a * x4
                + 3 * x5
                + (a + 3 * x) * y4
                + 2 * (a3 + 3 * a2 * x + a * x2+ 3 * x3) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a6 * nu
            - pi * a6
            + (pi * a2 * nu - pi * a2) * x4
            + (pi * a2 * nu - pi * a2) * y4
            - 2 * (pi * a4 * nu - pi * a4) * x2
            + 2
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y2
        )
    )

    # f5
    f[5, 0, :] = (
        3
        / 32
        * (
            8 * a8
            - 24 * a7 * x
            - 16 * a6 * x2
            + 60 * a5 * x3
            + 8 * a4 * x4
            - 48 * a3 * x5
            + 12 * a * x7
            + 12 * a * x * y6
            + 4 * (2 * a4 + 12 * a3 * x + 9 * a * x3) * y4
            + 4
            * (4 * a6 + 3 * a5 * x - 12 * a4 * x2+ 9 * a * x5)
            * y2
            - 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )

    f[5, 1, :] = (
        1
        / 16
        * (
            56 * a7 * x
            - 148 * a5 * x3
            + 128 * a3 * x5
            - 36 * a * x7
            - 36 * a * x * y6
            - 12 * (8 * a3 * x + 9 * a * x3) * y4
            - 4 * (a5 * x - 8 * a3 * x3+ 27 * a * x5) * y2
            + 9
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 9
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )

    f[5, 2, :] = (
        -3
        / 32
        * (
            8 * a8
            + 24 * a7 * x
            - 16 * a6 * x2
            - 60 * a5 * x3
            + 8 * a4 * x4
            + 48 * a3 * x5
            - 12 * a * x7
            - 12 * a * x * y6
            + 4 * (2 * a4 - 12 * a3 * x - 9 * a * x3) * y4
            + 4
            * (4 * a6 - 3 * a5 * x - 12 * a4 * x2- 9 * a * x5)
            * y2
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 + 2 * a * x + x2+ y2))
            - 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
                + y8
                + 4 * (a2 + x2) * y6
                + 2 * (3 * a4 + 2 * a2 * x2+ 3 * x4) * y4
                + 4 * (a6 - a4 * x2- a2 * x4 + x6) * y2
            )
            * np.log(abs(a2 - 2 * a * x + x2+ y2))
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )

    # f6
    f[6, 0, :] = (
        -3
        / 16
        * (
            3 * y8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a * y7
            + 12
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y6
            - 6 * (4 * a3 + 3 * a * x2) * y5
            + 6
            * (
                (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_plus_a
                + (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_minus_a
            )
            * y4
            - 2 * (15 * a5 - 8 * a4 * x + 9 * a * x4) * y3
            + 12
            * (
                (a6 - a4 * x2- a2 * x4 + x6) * arctan_x_plus_a
                + (a6 - a4 * x2- a2 * x4 + x6)
                * arctan_x_minus_a
            )
            * y2
            - 2
            * (
                6 * a7
                - 8 * a6 * x
                + 3 * a5 * x2
                + 8 * a4 * x3
                - 12 * a3 * x4
                + 3 * a * x6
            )
            * y
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_plus_a
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_minus_a
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )

    f[6, 1, :] = (
        1
        / 8
        * (
            9 * y8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 18 * a * y7
            + 36
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y6
            - 2 * (32 * a3 + 27 * a * x2) * y5
            + 18
            * (
                (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_plus_a
                + (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_minus_a
            )
            * y4
            - 2 * (37 * a5 + 8 * a3 * x2+ 27 * a * x4) * y3
            + 36
            * (
                (a6 - a4 * x2- a2 * x4 + x6) * arctan_x_plus_a
                + (a6 - a4 * x2- a2 * x4 + x6)
                * arctan_x_minus_a
            )
            * y2
            - 2
            * (14 * a7 + a5 * x2- 24 * a3 * x4 + 9 * a * x6)
            * y
            + 9
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_plus_a
            + 9
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_minus_a
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )

    f[6, 2, :] = (
        -3
        / 16
        * (
            3 * y8 * (arctan_x_plus_a + arctan_x_minus_a)
            - 6 * a * y7
            + 12
            * (
                (a2 + x2) * arctan_x_plus_a
                + (a2 + x2) * arctan_x_minus_a
            )
            * y6
            - 6 * (4 * a3 + 3 * a * x2) * y5
            + 6
            * (
                (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_plus_a
                + (3 * a4 + 2 * a2 * x2+ 3 * x4) * arctan_x_minus_a
            )
            * y4
            - 2 * (15 * a5 + 8 * a4 * x + 9 * a * x4) * y3
            + 12
            * (
                (a6 - a4 * x2- a2 * x4 + x6) * arctan_x_plus_a
                + (a6 - a4 * x2- a2 * x4 + x6)
                * arctan_x_minus_a
            )
            * y2
            - 2
            * (
                6 * a7
                + 8 * a6 * x
                + 3 * a5 * x2
                - 8 * a4 * x3
                - 12 * a3 * x4
                + 3 * a * x6
            )
            * y
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_plus_a
            + 3
            * (
                a8
                - 4 * a6 * x2
                + 6 * a4 * x4
                - 4 * a2 * x6
                + x8
            )
            * arctan_x_minus_a
        )
        / (
            pi * a10 * nu
            - pi * a10
            + (pi * a2 * nu - pi * a2) * x8
            + (pi * a2 * nu - pi * a2) * y8
            - 4 * (pi * a4 * nu - pi * a4) * x6
            + 4
            * (
                pi * a4 * nu
                - pi * a4
                + (pi * a2 * nu - pi * a2) * x2
            )
            * y6
            + 6 * (pi * a6 * nu - pi * a6) * x4
            + 2
            * (
                3 * pi * a6 * nu
                - 3 * pi * a6
                + 3 * (pi * a2 * nu - pi * a2) * x4
                + 2 * (pi * a4 * nu - pi * a4) * x2
            )
            * y4
            - 4 * (pi * a8 * nu - pi * a8) * x2
            + 4
            * (
                pi * a8 * nu
                - pi * a8
                + (pi * a2 * nu - pi * a2) * x6
                - (pi * a4 * nu - pi * a4) * x4
                - (pi * a6 * nu - pi * a6) * x2
            )
            * y2
        )
    )
    return f

In [54]:
def original_f_loop(x, y, a, nu):
    for i in range(0, np.int(x.size/10)):
        xt = x[i*10: (i+1)*10]
        yt = x[i*10: (i+1)*10]
        original_f_vector(xt, yt, a, nu)

In [55]:
%timeit original_f_loop(x, y, a, nu)
%timeit original_f_vector(x, y, a, nu)
%timeit original_f_vector_sub(x, y, a, nu)

1.85 s ± 52.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
69.2 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


TypeError: unhashable type: 'numpy.ndarray'